# Notebook to test the predictions

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import sys
from pathlib import Path

ML_DIR = Path().resolve().parents[0]
sys.path.append(str(ML_DIR))

# Import ML functions
from update_rolling import update_rolling_stats
from predict import predict_points

load_dotenv()
DATABASE_URL = os.getenv("ML_DATABASE_URL")
engine = create_engine(DATABASE_URL)


In [3]:
df_latest_rolling = update_rolling_stats(engine)

print("Latest rolling rows per player:")
display(df_latest_rolling.head())


No new games to update.
Latest rolling rows per player:


,player_id,game_id,game_date,matchup,team_abbreviation,minutes,points,assists,rebounds,steals,blocks,turnovers,avg_points_last5,avg_assists_last5,avg_rebounds_last5,avg_minutes_last5
0,2544,22500607,2026-01-18,LAL vs. TOR,LAL,32.0,24.0,7.0,4.0,0.0,2.0,3.0,25.2,6.8,7.0,32.6
1,101108,22500313,2025-12-01,LAC @ MIA,NaN,15.0,8.0,3.0,0.0,0.0,0.0,1.0,3.8,2.4,1.2,13.4
2,200768,22500560,2026-01-12,PHI @ TOR,PHI,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,1.2,0.6,8.2
3,201142,22500591,2026-01-16,HOU vs. MIN,HOU,40.0,39.0,7.0,4.0,2.0,1.0,5.0,27.8,4.8,7.4,38.8
4,201143,22500584,2026-01-15,GSW vs. NYK,GSW,21.0,5.0,5.0,5.0,1.0,2.0,1.0,6.6,3.8,6.0,18.4


## Run predictions

In [13]:
df_preds = predict_points(engine)

print("Predictions:")
df_preds = df_preds.sort_values("game_date", ascending=False)
display(df_preds.head(20))


C:\Users\kalz9\OneDrive\Documents\onedrive\Documents\Gamblr\gamblr\ml\predict.py:31: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  


Predictions:


,player_id,team_abbreviation,matchup,game_date,pred_points
0,1626192,CHA,CLE @ CHA,2026-01-21,3.947762
80,202699,DET,DET @ NOP,2026-01-21,11.626084
93,203914,MIL,OKC @ MIL,2026-01-21,2.419541
92,203648,MIL,OKC @ MIL,2026-01-21,2.067574
91,203507,MIL,OKC @ MIL,2026-01-21,28.150015
90,1642450,DET,DET @ NOP,2026-01-21,8.145899
89,1642404,DET,DET @ NOP,2026-01-21,3.868184
88,1642403,DET,DET @ NOP,2026-01-21,3.672035
87,1641842,DET,DET @ NOP,2026-01-21,7.487670
86,1641752,DET,DET @ NOP,2026-01-21,4.113140
